In [0]:
dbutils.widgets.text("name","","name")
name = dbutils.widgets.get("name")
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW file_history AS
SELECT
    h.table_name,
    h.version,
    h.timestamp,
    h.operation,
    h.userName,
    f.file_path AS file
FROM edsm.silver.systemspopulated_transaction_history h
JOIN (
    SELECT
        primary_key,
        EXPLODE(file_path) AS file_path
    FROM edsm.silver.systemspopulated_file_version_history
) f
  ON h.primary_key = f.primary_key;
""")
spark.sql(f"""
    SELECT
        fh.*
    FROM edsm.silver.systemspopulated s
    JOIN file_history fh
    ON s.file_path = fh.file
    WHERE s.name = '{name}'
    ORDER BY fh.timestamp
""").display()